In [17]:
%pip install python-dotenv
%pip install pandas
%pip install boto3
%pip install langfuse
%pip install gitpython


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import dotenv
dotenv.load_dotenv("../local.env")
import os
import sys
sys.path.append(
    os.path.join(os.path.dirname(os.path.abspath('.')), "python/lambda")
)
import pandas as pd
from services.openai_client import OpenAIClient
from langfuse.decorators import observe, langfuse_context
openai_client = OpenAIClient(
        os.environ["OPENAI_API_KEY"], os.environ["LANGFUSE_KEY"]
)
from services.neptune_client import NeptuneClient
import re
neptune_client = NeptuneClient("https://127.0.0.1:8182")
import git
repo = git.Repo(search_parent_directories=True)
sha = repo.head.object.hexsha
from langfuse import Langfuse
langfuse = Langfuse(
  secret_key=os.environ["LANGFUSE_KEY"],
  public_key="pk-lf-b2888d04-2d31-4b07-8f53-d40d311d4d13",
  host="https://cloud.langfuse.com",
  release="local_"+sha
)
import json

In [ ]:
pf = pd.read_csv("get_article_summarization.csv")

In [ ]:
pf.head()

In [ ]:
import json
input = json.loads(pf.iloc[5].input)
from pprint import pprint
pprint(input)
pf.iloc[5].output
len(input['args'][0])

In [ ]:

new_prompt = """
Your task is to identity the entities and relations that are the subject of the following text. use dbpedia ontologies to model the entities and relations. Output in turtle rdf format.
---
"""
import hashlib

def hash_prompt(prompt):
    return hashlib.md5(prompt.encode()).hexdigest()

prompt_hash = hash_prompt(new_prompt)
print(f"Hash of new_prompt: {prompt_hash}")

@observe(name="get_article_summarization")
def get_article_summarization(input):
    langfuse_context.update_current_trace(
        tags=["experimentation", "notebook", prompt_hash]

    )
    return openai_client.get_completion(new_prompt, input,model="gpt-4o")


In [ ]:

open_cypher = openai_client.get_article_graph(input['args'][0],article_id="some_uuid")
open_cypher = re.sub(r"date\(('|\".*?'|\")\)", r"\1", open_cypher)
print(open_cypher)
res = neptune_client.query(open_cypher)
print(res)

In [2]:
import boto3
from article_repo import ArticleRepository
from browse_repo import BrowseRepository
from repos import BrowsedRepository
from services.articles_service import ArticlesService
from services.themes_service import ThemesService
from theme_repo import ThemeRepository
secretsmanager = boto3.client("secretsmanager")
get_secret_value_response = secretsmanager.get_secret_value(
    SecretId=os.getenv("DB_SECRET_ARN")
)
secret = json.loads(get_secret_value_response["SecretString"])
article_repo = ArticleRepository(
    secret["username"],
    secret["password"],
    "dassie",
    os.getenv("DB_CLUSTER_ENDPOINT"),
)
browse_repo = BrowseRepository(
    secret["username"],
    secret["password"],
    "dassie",
    os.getenv("DB_CLUSTER_ENDPOINT"),
)
browsed_repo = BrowsedRepository(
    secret["username"],
    secret["password"],
    "dassie",
    os.getenv("DB_CLUSTER_ENDPOINT"),
)
theme_repo = ThemeRepository(
    secret["username"],
    secret["password"],
    "dassie",
    os.getenv("DB_CLUSTER_ENDPOINT"),
)
openai_client = OpenAIClient(
    os.environ["OPENAI_API_KEY"], os.environ["LANGFUSE_KEY"]
)
articles_service = ArticlesService(article_repo=article_repo, theme_repo=None, browse_repo=browse_repo, browsed_repo=browsed_repo, openai_client=openai_client)
themes_service = ThemesService(theme_repo=theme_repo, article_repo=article_repo, openai_client=openai_client)
@observe(name="article_graph_test")
def article_graph_test(article, article_id, tag):
    langfuse_context.update_current_trace(
        tags=["experimentation", "notebook", tag]

    )
    open_cypher = openai_client.get_article_graph(article.text,article_id=article_id)
    print(open_cypher)
    try:
        res = neptune_client.query(open_cypher)
        print(res)
    except Exception as e:
        print(e)
        langfuse_context.update_current_trace(
        tags=["neptune_exception"]
    )

In [12]:
in_q = "'"
for article in articles:
    in_q += str(article.id) + "','"
in_q = in_q[:-2]
print(in_q)
print(theme.id)
print(theme.title)

'a2452062-4748-4643-9856-410f3545d66d','79b248ee-240b-4342-8071-2ae09dbad22c','3d04cd9d-de07-49e7-be4b-78b3413f7284','937180be-5226-41ca-985d-9279acef87c0','5ef4761e-a240-4724-9ce2-faca2ce40696','6d1b6499-5130-4b99-b33f-8c8c6c0d9bb7','147d7657-9f56-4258-bdbc-b8ed27bd0346','6ddd4e93-573d-4443-927f-36066be16c5e','afd929d3-ef78-4e10-94ec-612e34f5cc73','e7d45e3d-103d-4469-bb98-418d9433f087','e407b921-f743-4e4e-9763-c4b395263379','5eeface4-a7ee-4d30-8124-6323e703f5f2','9d06ab54-9a16-4511-a434-16b364df8543','61cae0ab-5592-4360-9bce-11b853455359','d7d16efe-3df8-4ca8-9351-68df0f774acf','d625bca5-2963-492a-9e60-b0cc79ca1d73','e0d1fe6b-2e7a-47a0-b123-5ff7b7b604c6','f5ea2a99-89f0-45c2-a582-958573bdddf6','b9a80b36-10f9-4d18-89ec-5ce140b60a90','44a557c5-64bd-467e-bc1b-cb5b009dc92f'
0e0bd7d3-22c2-4e8f-8bfc-96d0a6a2fbb9
flying+to+majorca


In [11]:
theme = themes_service.get_theme_by_title("flying+to+majorca")
articles = theme.related
#article_graph_test(articles[0],articles[0].id,"theme_related")
#for article in articles:
#    article_graph_test(article,article.id,"theme_related")



In [ ]:
articles = articles_service._article_repo.get(limit=100,sort_by="logged_at",descending=True)
for article in articles:
    article_graph_test(article,article.id)

In [ ]:


obs = langfuse.fetch_observation("8cf07c1b-548a-43d1-a00a-81f1f2caac9f")
print(obs)
dir(obs)



In [9]:

traces = langfuse.fetch_traces(name="article_graph_test",tags=["neptune_exception"],limit=100)
for trace in traces.data:
    for obs in trace.observations:
        obs = langfuse.fetch_observation(obs)
        if obs.data.name == "get_article_graph":
            print(obs.data.trace_id)
            print(obs.data.id)
            try:
                query = obs.data.output
                if query.startswith(" provided text"):
                    query = openai_client._get_opencypher_code_block(query)
                res = neptune_client.query(query)
                print(res)
                article_id = obs.data.input['kwargs']['article_id']
                article = articles_service._article_repo.get_by_id(article_id)
                article_graph_test(article,article_id,"neptune_exception_fixed")
            except Exception as e:
                print(e)
            print("---\n\n\n")






97299858-810a-422d-81df-fc385583b1e7
8cf07c1b-548a-43d1-a00a-81f1f2caac9f
{"message":"Neptune query: \nMERGE (article:Article {id: '2bae4cab-b9e3-4028-ba38-e21fcaeaaabe'})\n\nMERGE (rdflib:RDFLib {name: 'RDFLib'})\nSET rdflib:Software,\n    rdflib.initialReleaseDate = '2002-06-04',\n    rdflib.latestReleaseVersion = '7.0.0',\n    rdflib.latestReleaseDate = '2022-07-16',\n    rdflib.description = \"RDFLib is a Python library for working with RDF, a simple yet powerful language for representing information. This library contains parsers/serializers for almost all of the known RDF serializations, such as RDF/XML, Turtle, N-Triples, & JSON-LD, many of which are now supported in their updated form.\",\n    rdflib.website = 'https://rdflib.readthedocs.io'\nMERGE (python:ProgrammingLanguage {name: 'Python (programming language)'})\nMERGE (rdflib)-[:DEVELOPER]->(python)\nMERGE (rdflib)-[:PROGRAMMING_LANGUAGE]->(python)\nMERGE (article)-[:MENTIONS]->(rdflib)\n\nMERGE (turtle:Turtle {name: 'Turt

/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:34:12,555+0100","service":"dassie"}
{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:34:12,610+0100","service":"dassie"}
{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:34:12,688+0100","service":"dassie"}
{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query

/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburk

{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:34:12,824+0100","service":"dassie"}
[[], [], [], [], [], [], []]

MERGE (article:Article {id: '2bae4cab-b9e3-4028-ba38-e21fcaeaaabe'})

MERGE (rdflib:RDFLib:Software {name: 'RDFLib'})
ON CREATE SET rdflib.initialReleaseDate = date('2002-06-04'), rdflib.latestReleaseVersion = '7.0.0', rdflib.latestReleaseDate = date('2023-10-11'), rdflib.description = 'RDFLib is a Python library for working with RDF, a simple yet powerful language for representing information. This library contains parsers/serializers for almost all of the known RDF serializations, such as RDF/XML, Turtle, N-Triples, & JSON-LD, many of which are now supported in their updated form.', rdflib.website = 'https://rdflib.readthedocs.io'
MERGE (python:ProgrammingLanguage {name: 'Python'})
MERGE (bsd

...

{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:58:02,313+0100","service":"dassie"}
{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:58:02,404+0100","service":"dassie"}


/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:58:02,657+0100","service":"dassie"}
[[], [], [], [], [], [], [], [], []]
---



a84325c3-8ce0-4fcd-8f05-ab93d7d5ae4a
7c46a642-2634-46b0-9b0b-73f9b787ffc4
{"message":"Neptune query: \nMERGE (article:Article {id: 'b8206f17-b948-42e9-a3d2-6898e0ac4122'})\n\nMERGE (python:ProgrammingLanguage {name: 'Python_(programming_language)'})\nON CREATE SET python.abstract = \"Python is a high-level, interpreted programming language known for its readability and versatility.\"\nMERGE (article)-[:MENTIONS]->(python)\n\nMERGE (google:Product {name: 'Google_for_Education'})\nMERGE (google)-[:RELATED_TO]->(python)\nMERGE (article)-[:MENTIONS]->(google)\n\nMERGE (regex:ProgrammingLanguageFeature {name: 'Regular_expression'})\nON CREATE SET regex.abstract = \"Regular expressions 

/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburk

{"message":"Error querying Neptune: An error occurred (BadRequestException) when calling the ExecuteOpenCypherQuery operation: Expected a simple literal but found CType.","level":"ERROR","location":"query:34","timestamp":"2024-10-01 11:58:03,221+0100","service":"dassie"}
An error occurred (BadRequestException) when calling the ExecuteOpenCypherQuery operation: Expected a simple literal but found CType.
---





In [5]:
obs = langfuse.fetch_observation('bc4b5a90-d1ba-45b9-ba06-3cdcb2a6156f')
print(obs.data.input['kwargs']['article_id'])

9e9e911d-9758-4fa6-b497-fe247ae6d896


In [4]:
obs = langfuse.fetch_observation('bc4b5a90-d1ba-45b9-ba06-3cdcb2a6156f')
print(obs.data.input['kwargs']['article_id'])
print(obs.data.trace_id)
print(obs.data.output)

res = neptune_client.query(obs.data.output)

print(res)
print("---\n\n\n")





713f83db-7cd3-4343-a8ee-53386b6ccf70

MERGE (a:Article {id: '9e9e911d-9758-4fa6-b497-fe247ae6d896'})

MERGE (awsNeptune:Software {name: 'AWS Neptune Analytics API'})
ON CREATE SET awsNeptune.developer = 'Amazon Web Services', awsNeptune.documentation = 'https://docs.aws.amazon.com/neptune/latest/userguide/analytics-api.html'
MERGE (awsNeptune)-[:MENTIONED_IN]->(a)

MERGE (resetGraph:Function {name: 'ResetGraph'})
ON CREATE SET resetGraph.description = 'Empties the data from a specified Neptune Analytics graph.'
MERGE (resetGraph)-[:PART_OF]->(awsNeptune)
MERGE (resetGraph)-[:MENTIONED_IN]->(a)

MERGE (graph:Dataset {identifier: 'graphIdentifier'})
ON CREATE SET graph.pattern = 'g-[a-z0-9]{10}'
MERGE (graph)-[:MENTIONED_IN]->(a)

MERGE (responseElements:Response {name: 'Response Elements'})
ON CREATE SET responseElements.statusCode = '200', responseElements.contentType = 'application/json'
MERGE (responseElements)-[:PART_OF]->(resetGraph)
MERGE (responseElements)-[:MENTIONED_IN]->(a)

M

/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburk

{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:29:43,958+0100","service":"dassie"}
{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:29:44,040+0100","service":"dassie"}
{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 11:29:44,115+0100","service":"dassie"}
{"message":"Error querying Neptune: An error occurred (MalformedQueryException) when calling the ExecuteOpenCypherQuery operation: Variable `responseElements` not defined (line 1, column 6 (offset: 5

/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


MalformedQueryException: An error occurred (MalformedQueryException) when calling the ExecuteOpenCypherQuery operation: Variable `responseElements` not defined (line 1, column 6 (offset: 5))

In [13]:
query = """
MERGE (a:Article {id: '9e9e911d-9758-4fa6-b497-fe247ae6d896'})

MERGE (awsNeptune:Software {name: 'AWS Neptune Analytics API'})
ON CREATE SET awsNeptune.developer = 'Amazon Web Services', awsNeptune.documentation = 'https://docs.aws.amazon.com/neptune/latest/userguide/analytics-api.html'
MERGE (awsNeptune)-[:MENTIONED_IN]->(a)

MERGE (resetGraph:Function {name: 'ResetGraph'})
ON CREATE SET resetGraph.description = 'Empties the data from a specified Neptune Analytics graph.'
MERGE (resetGraph)-[:PART_OF]->(awsNeptune)
MERGE (resetGraph)-[:MENTIONED_IN]->(a)

MERGE (graph:Dataset {identifier: 'graphIdentifier'})
ON CREATE SET graph.pattern = 'g-[a-z0-9]{10}'
MERGE (graph)-[:MENTIONED_IN]->(a)

MERGE (responseElements:Response {name: 'Response Elements'})
ON CREATE SET responseElements.statusCode = '200', responseElements.contentType = 'application/json'
MERGE (responseElements)-[:PART_OF]->(resetGraph)
MERGE (responseElements)-[:MENTIONED_IN]->(a)
MERGE (errors:Error {name: 'Errors'})
MERGE (errors)-[:PART_OF]->(resetGraph)
MERGE (errors)-[:MENTIONED_IN]->(a)
WITH responseElements
UNWIND [
  {property: 'arn', type: 'String'},
  {property: 'buildNumber', type: 'String'},
  {property: 'createTime', type: 'Timestamp'},
  {property: 'deletionProtection', type: 'Boolean'},
  {property: 'endpoint', type: 'String'},
  {property: 'id', type: 'String', pattern: 'g-[a-z0-9]{10}'},
  {property: 'kmsKeyIdentifier', type: 'String', pattern: 'arn:aws(|-cn|-us-gov):kms:[a-zA-Z0-9-]*:[0-9]{12}:key/[a-zA-Z0-9-]{36}'},
  {property: 'name', type: 'String', pattern: '(?!g-)[a-z][a-z0-9]*(-[a-z0-9]+)*'},
  {property: 'provisionedMemory', type: 'Integer', validRange: '32-24576'},
  {property: 'publicConnectivity', type: 'Boolean'},
  {property: 'replicaCount', type: 'Integer', validRange: '0-2'},
  {property: 'sourceSnapshotId', type: 'String', pattern: 'gs-[a-z0-9]{10}'},
  {property: 'status', type: 'String', validValues: 'CREATING, AVAILABLE, DELETING, RESETTING, UPDATING, SNAPSHOTTING, FAILED, IMPORTING'},
  {property: 'statusReason', type: 'String'},
  {property: 'vectorSearchConfiguration', type: 'VectorSearchConfiguration', maxDimensions: '65535'}
] AS element
MERGE (prop:Property {name: element.property})
ON CREATE SET prop.type = element.type, prop.pattern = element.pattern, prop.validRange = element.validRange, prop.validValues = element.validValues, prop.maxDimensions = element.maxDimensions
MERGE (responseElements)-[:CONTAINS]->(prop)
WITH errors
UNWIND [
  {property: 'ConflictException', statusCode: '409'},
  {property: 'InternalServerException', statusCode: '500'},
  {property: 'ResourceNotFoundException', statusCode: '404'},
  {property: 'ThrottlingException', statusCode: '429'},
  {property: 'ValidationException', statusCode: '400'}
] AS error
MERGE (err:ErrorProperty {name: error.property})
ON CREATE SET err.statusCode = error.statusCode
MERGE (errors)-[:CONTAINS]->(err)
"""
neptune_client.query(query)

{"message":"Neptune query: \nMERGE (a:Article {id: '9e9e911d-9758-4fa6-b497-fe247ae6d896'})\n\nMERGE (awsNeptune:Software {name: 'AWS Neptune Analytics API'})\nON CREATE SET awsNeptune.developer = 'Amazon Web Services', awsNeptune.documentation = 'https://docs.aws.amazon.com/neptune/latest/userguide/analytics-api.html'\nMERGE (awsNeptune)-[:MENTIONED_IN]->(a)\n\nMERGE (resetGraph:Function {name: 'ResetGraph'})\nON CREATE SET resetGraph.description = 'Empties the data from a specified Neptune Analytics graph.'\nMERGE (resetGraph)-[:PART_OF]->(awsNeptune)\nMERGE (resetGraph)-[:MENTIONED_IN]->(a)\n\nMERGE (graph:Dataset {identifier: 'graphIdentifier'})\nON CREATE SET graph.pattern = 'g-[a-z0-9]{10}'\nMERGE (graph)-[:MENTIONED_IN]->(a)\n\nMERGE (responseElements:Response {name: 'Response Elements'})\nON CREATE SET responseElements.statusCode = '200', responseElements.contentType = 'application/json'\nMERGE (responseElements)-[:PART_OF]->(resetGraph)\nMERGE (responseElements)-[:MENTIONED_IN

/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Users/davidburk

{"message":"Neptune query response: {'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked'}, 'RetryAttempts': 0}, 'results': []}","level":"INFO","location":"query:26","timestamp":"2024-10-01 10:34:39,190+0100","service":"dassie"}
{"message":"Error querying Neptune: An error occurred (MalformedQueryException) when calling the ExecuteOpenCypherQuery operation: Variable `errors` not defined (line 29, column 6 (offset: 1771))","level":"ERROR","location":"query:34","timestamp":"2024-10-01 10:34:39,239+0100","service":"dassie"}


/Users/davidburke/src/dassie/.venv/lib/python3.9/site-packages/urllib3/connectionpool.py:1063: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


MalformedQueryException: An error occurred (MalformedQueryException) when calling the ExecuteOpenCypherQuery operation: Variable `errors` not defined (line 29, column 6 (offset: 1771))

In [ ]:
query = """MERGE (a:Article {id: 'b3c845a6-60b5-40cc-838a-e86043d41602'})

MERGE (wp1:WebPage {url: 'https://platform.openai.com/docs/models'})
SET wp1.title = 'Models - OpenAI API',
    wp1.description = "Explore resources, tutorials, API docs, and dynamic examples to get the most out of OpenAI's developer platform."
MERGE (wp1)-[:SOURCE_OF]->(a)

MERGE (c:Company {name: 'OpenAI'})
MERGE (c)-[:SOURCE_OF]->(a)

MERGE (s1:Software {name: 'GPT-4o'})
SET s1.description = 'Our high-intelligence flagship model'
MERGE (s1)-[:SOURCE_OF]->(a)
MERGE (c)-[:PRODUCT]->(s1)

MERGE (wp2:WebPage {url: 'https://learn.microsoft.com/Learn/Azure/AI_Services'})
SET wp2.title = 'Azure OpenAI Service models',
    wp2.description = 'Azure OpenAI Service is powered by a diverse set of models with different capabilities and price points. Model availability varies by region.'
MERGE (wp2)-[:SOURCE_OF]->(a)

MERGE (wp3:WebPage {url: 'https://openai.com/form/custom-models'})
SET wp3.title = 'Custom models',
    wp3.description = 'The Custom Models program gives selected organizations an opportunity to work with a dedicated group of OpenAI researchers to train custom GPT-4 models.'
MERGE (wp3)-[:SOURCE_OF]->(a)

MERGE (wp4:WebPage {url: 'https://openai.com/index/introducing-the-model-spec'})
SET wp4.title = 'Introducing the Model Spec',
    wp4.date = date("2024-05-08"),
    wp4.description = 'We are sharing a first draft of the Model Spec, a new document that specifies how we want our models to behave in the OpenAI API and ChatGPT.'
MERGE (wp4)-[:SOURCE_OF]->(a)

MERGE (wp5:WebPage {url: 'https://openai.com/index/new-models-and-developer-products-announced-at-DevDay'})
SET wp5.title = 'New models and developer products announced at DevDay',
    wp5.date = date('2023-11-06'),
    wp5.description = 'GPT-4 Turbo with 128K context and lower prices, the new Assistants API, GPT-4 Turbo with Vision, DALL·E 3 API, and more.'
MERGE (wp5)-[:SOURCE_OF]->(a)"""
import re
open_cypher = re.sub(r"date\(('|\".*?'|\")\)", r"\1", query)
print(open_cypher)

In [ ]:
df = pf.iloc[0:10]
for index, row in df.iterrows():
    input = json.loads(row.input)
    output = get_article_summarization(input['args'][0])
    df.loc[index, "output"] = output

In [ ]:
df.iloc[5].output
new_prompt = """
Your task is to validate that each of the following entities and relations in the turtle are grounded in the text. output opencypher (Neptune-9.0.20190305-1.0) query to create only the grounded entities and relations, assume the entities and relations may already exist. Include 
an Article entity with id {article_id} that relates to all the other entities as the source of the information.
---
"""
prompt_hash = hash_prompt(new_prompt)
print(f"Hash of new_prompt: {prompt_hash}")
@observe(name="get_article_summarization_validate")
def get_article_summarization_validate(input,turtle,article_id):
    langfuse_context.update_current_trace(
        tags=["experimentation", "notebook", prompt_hash]

    )
    return openai_client.get_completion(new_prompt.format(article_id=article_id), turtle+"---"+input,model="gpt-4o")

print(get_article_summarization_validate(json.loads(df.iloc[5].input)['args'][0],df.iloc[5].output,article_id="some_uuid"))



In [ ]:
df = pf.iloc[0:10]
for index, row in df.iterrows():
    input = json.loads(row.input)
    output =get_article_summarization_validate(json.loads(row.input)['args'][0],row.output) 
    df.loc[index, "output_validate"] = output
    print(json.loads(row.input)['args'][0])
    print(row.output)
    print(output)
    print("---\n\n\n")



In [ ]:
format_prompt = """
Your task is to translate the entities and relationships described in the following turtle triplets to json-ld that complies schemas from schema.org only.
Output strictly valid json-ld, that complies with the selected schema.org vocabularies. Do not include any additional text or formatting.If the entity or relationship is not in schema.org, leave it out of the output.
---
"""

format_prompt_hash = hash_prompt(format_prompt)
print(f"Hash of format_prompt: {format_prompt_hash}")

@observe(name="get_article_summarization_json_ld")
def get_article_summarization_json_ld(input):
    langfuse_context.update_current_trace(
        tags=["experimentation", "notebook", format_prompt_hash]

    )
    return openai_client.get_completion(format_prompt, input)

    

In [14]:
traces = langfuse.fetch_traces(name="article_graph_test",tags=["neptune_exception_fixed"],limit=100)
for trace in traces.data:
    for obs in trace.observations:
        obs = langfuse.fetch_observation(obs)
        if obs.data.name == "get_article_graph":
            langfuse.create_dataset_item(
                dataset_name="get_article_graph",
                metadata={"tags":["neptune_exception_fixed"]},
                source_trace_id=obs.data.trace_id,
                source_observation_id=obs.data.id,
                input=obs.data.input,
                expected_output=obs.data.output,
            )
            

MERGE (article:Article {id: 'e6389c98-77ed-48e0-8f0d-6717635f0e64'}) MERGE (gremlinInc:Company {name: 'Gremlin Inc.'}) MERGE (stackOverflow:Website {name: 'Stack Overflow'}) MERGE (janusGraph:Software {name: 'JanusGraph'}) MERGE (apacheTinkerPop:Software {name: 'Apache TinkerPop'}) MERGE (amazonWebServices:Company {name: 'Amazon Web Services'}) MERGE (googleGroups:Website {name: 'Google Groups'}) MERGE (gremlinQueryLanguage:ProgrammingLanguage {name: 'Gremlin'}) MERGE (azureCosmosDB:Database {name: 'Azure Cosmos DB'}) MERGE (kubernetes:Software {name: 'Kubernetes'}) MERGE (chaosEngineering:EngineeringDiscipline {name: 'Chaos engineering'}) MERGE (devOps:EngineeringDiscipline {name: 'DevOps'}) MERGE (webPage1:WebPage {url: 'https://www.gremlin.com/getting-started-authentication', title: 'Getting Started > Authenticating the Gremlin Agent'}) MERGE (webPage1)-[:PUBLISHED_BY]->(gremlinInc) MERGE (article)-[:MENTIONS]->(webPage1) MERGE (webPage2:WebPage {url: 'https://stackoverflow.com/ques

In [16]:
%env

{'AWS_CONTAINER_AUTHORIZATION_TOKEN': '<hidden>',
 'AWS_CONTAINER_CREDENTIALS_FULL_URI': 'http://127.0.0.1:50220',
 'AWS_DEFAULT_REGION': 'eu-west-1',
 'AWS_REGION': 'eu-west-1',
 'AWS_VAULT': 'dburke',
 'COMMAND_MODE': 'unix2003',
 'CONDA_EXE': '/Users/davidburke/miniforge3/bin/conda',
 'CONDA_PYTHON_EXE': '/Users/davidburke/miniforge3/bin/python',
 'CONDA_SHLVL': '0',
 'ELECTRON_NO_ATTACH_CONSOLE': '1',
 'HOME': '/Users/davidburke',
 'HOMEBREW_CELLAR': '/opt/homebrew/Cellar',
 'HOMEBREW_PREFIX': '/opt/homebrew',
 'HOMEBREW_REPOSITORY': '/opt/homebrew',
 'INFOPATH': '/opt/homebrew/share/info:',
 'LANG': 'en_IE.UTF-8',
 'LOGNAME': 'davidburke',
 'MANPATH': '/Users/davidburke/.nvm/versions/node/v22.2.0/share/man:/opt/homebrew/share/man::',
 'MallocNanoZone': '0',
 'NVM_BIN': '/Users/davidburke/.nvm/versions/node/v22.2.0/bin',
 'NVM_CD_FLAGS': '-q',
 'NVM_DIR': '/Users/davidburke/.nvm',
 'NVM_INC': '/Users/davidburke/.nvm/versions/node/v22.2.0/include/node',
 'ORIGINAL_XDG_CURRENT_DESKTO